<a href="https://colab.research.google.com/github/NeoLin1103/104_web_scraping/blob/main/104_web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# install chromium, its driver, and selenium
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install tqdm

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [69.5 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [5]:
from selenium import webdriver
# set options to be headless, ..
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

options.add_argument("start-maximized")
options.add_argument("enable-automation")
options.add_argument("--disable-infobars")
options.add_argument("--disable-browser-side-navigation")
options.add_argument("--disable-gpu")

In [6]:
from bs4 import BeautifulSoup
import pandas as pd
import requests as rq
import time
from tqdm import tqdm, trange

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
"""
1. 可以嘗試使用selenium從主頁搜尋關鍵字"數據分析"
2. 自動獲得總頁數資料
3. webdriver打開子頁速度很慢，有沒有更好的方法?
* 修正排版 (統一變數名稱)
"""

In [ ]:
"""
職稱和公司名稱
薪資
工作內容
擅長工具和工作技能
其他條件
"""

In [7]:
urls = []

#第一頁的前十個是廣告不能用
url = f"https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=%E6%95%B8%E6%93%9A%E5%88%86%E6%9E%90&expansionType=job&order=15&asc=0&page={1}&mode=s&jobsource=cmw_redirect"
request = rq.get(url)
soup = BeautifulSoup(request.text, 'lxml') 
#主頁中的各職缺訊息
articles = soup.find_all("article")
for j in range(10,30):
  #獲得各職缺子頁的url
  url = articles[j].find("a", {"class": "js-job-link"})["href"]
  urls.append(url)

In [8]:
for i in trange(2,100):
  #進入主頁，並進行換頁
  url = f"https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=%E6%95%B8%E6%93%9A%E5%88%86%E6%9E%90&expansionType=job&order=15&asc=0&page={i}&mode=s&jobsource=cmw_redirect"
  request = rq.get(url)
  soup = BeautifulSoup(request.text, 'lxml') 
  #主頁中的各職缺訊息
  articles = soup.find_all("article")
  for j in range(20):
    #獲得各職缺子頁的url
    url = articles[j].find("a", {"class": "js-job-link"})["href"]
    urls.append(url)
  time.sleep(1)

100%|██████████| 98/98 [04:28<00:00,  2.74s/it]


In [ ]:
print(len(urls))

180


In [9]:
jobs_scraper = {}

In [10]:
#薪資
#確定一下為什麼一頁變成24篇 (是因為webdriver跟request的差別? (不是)) (第一頁有32篇，後面每一頁是22篇)
#每一頁最後兩個是不需要的
#第一頁前十個是廣告
articles = []

#第一頁的前十個是廣告不能用
url = f"https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=%E6%95%B8%E6%93%9A%E5%88%86%E6%9E%90&expansionType=job&order=15&asc=0&page={1}&mode=s&jobsource=cmw_redirect"
request = rq.get(url)
soup = BeautifulSoup(request.text, 'lxml') 
#主頁中的各職缺訊息
article = soup.find_all("article")
for j in range(10,30):
  articles.append(article[j])

In [11]:
for page in trange(2,100):
  url = f"https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=%E6%95%B8%E6%93%9A%E5%88%86%E6%9E%90&expansionType=job&order=15&asc=0&page={page}&mode=s&jobsource=cmw_redirect"
  request = rq.get(url)
  soup = BeautifulSoup(request.text, 'lxml') 
  article = soup.find_all("article")
  for i in range(20):
    articles.append(article[i])
  time.sleep(1)

100%|██████████| 98/98 [04:31<00:00,  2.77s/it]


In [12]:
jobs_scraper["Salary"] = []
for i in trange(len(articles)):
  #從主頁得到的各職缺訊息即可得到薪資資料
  salary = articles[i].find("span", {"class":"b-tag--default"})
  jobs_scraper["Salary"].append(salary.text)

100%|██████████| 1980/1980 [00:00<00:00, 6849.52it/s]


In [13]:
print(len(jobs_scraper["Salary"]))

1980


In [ ]:
print(jobs_scraper["Salary"][:30])

['待遇面議', '月薪40,000~60,000元', '待遇面議', '待遇面議', '待遇面議', '月薪38,000元以上', '待遇面議', '月薪64,000~78,000元', '待遇面議', '待遇面議', '月薪40,000~50,000元', '待遇面議', '待遇面議', '待遇面議', '待遇面議', '月薪32,000~60,000元', '待遇面議', '待遇面議', '月薪34,000~45,000元', '待遇面議', '待遇面議', '待遇面議', '待遇面議', '月薪40,000元以上', '待遇面議', '月薪26,000~40,000元', '月薪34,000元以上', '月薪35,000~41,000元', '月薪30,000~40,000元', '月薪35,000~42,000元']


In [52]:
#職稱和公司名稱
#要確定不同項目的順序有對得起來 (除了薪資以外都是使用相同的網址list)
#排除無關的搜尋結果 (已扣除廣告)
jobs_scraper["Position"] = []
jobs_scraper["Company"] = []
for i in trange(len(urls)):
  url = "http:" + urls[i]
  request = rq.get(url)
  soup = BeautifulSoup(request.text, 'lxml')
  #從各職缺子頁的title得到資料
  string_list = soup.title.text.split("｜")
  if len(string_list) > 1:
    jobs_scraper["Position"].append(string_list[0])
    jobs_scraper["Company"].append(string_list[1])
  else:
    jobs_scraper["Position"].append("")
    jobs_scraper["Company"].append("")
  time.sleep(1)

100%|██████████| 1980/1980 [1:10:32<00:00,  2.14s/it]


In [51]:
i = 373
url = "http:" + urls[i]
request = rq.get(url)
soup = BeautifulSoup(request.text, 'lxml')
print(soup.title)

<title>職缺已關閉－104 人力銀行</title>


In [ ]:
print(jobs_scraper["Position"][:30])
print(jobs_scraper["Company"][:30])

['數據分析人員', '數據分析專員', '數據分析師', '數據分析人員', '數據分析師', '數據分析專案副理', 'Google Analytics 資深數據分析師', 'Data Analysis-數據分析產品經理', '資深會計暨數據分析管理師', '資訊-智慧數據分析工程師', '(內湖總公司)數據分析專案副理', '數據分析師/主管', '【台中市西屯區】 業務數據分析師', '數據分析師', '行銷數據分析師【明基BenQ雲端創新中心】', '數位行銷數據分析師', '【策略分析部】數據分析師', '數據分析顧問(B216)', 'AI數據分析工程師', 'EN10026 品質數據分析工程師']
['(總公司)南山人壽保險股份有限公司', '(總公司)南山人壽保險股份有限公司', 'Royce Tech Ltd', '神揚保險代理人股份有限公司', '台北神策諮詢顧問有限公司', '特力集團_特力股份有限公司', '英屬開曼群島商雲數位智能行銷有限公司台灣分公司', '替您錄科技股份有限公司(TGC Taiwan, Inc.)', '春霖國際股份有限公司', '臻鼎科技股份有限公司', '特力集團＿HOLA＿HOLA CASA＿HOLA Petite＿Crate&Barrel＿特力屋股份有限公司', '華騰企管顧問有限公司', '香港商鈞陽有限公司台灣辦事處', '弈樂科技股份有限公司', '明基電通股份有限公司', '台灣松下電器股份有限公司', '聯合智網股份有限公司', '精誠資訊股份有限公司', '科智企業股份有限公司', '華碩電腦股份有限公司']


In [ ]:
#其他條件
jobs_scraper["Other_requirements"] = []
for i in trange(len(urls)):
  url = "http:" + urls[i]
  driver = webdriver.Chrome(options=options) #用webdriver才有讀到
  driver.get(url)
  soup = BeautifulSoup(driver.page_source, 'html.parser')
  requirement = soup.find("p",{"class":"m-0 r3 w-100"})
  if requirement == None:
    jobs_scraper["Other_requirements"].append(" ")
  else:
    jobs_scraper["Other_requirements"].append(soup.find("p",{"class":"m-0 r3 w-100"}).text)
  driver.quit()

100%|██████████| 180/180 [16:32<00:00,  5.51s/it]


In [ ]:
print(jobs_scraper["Other_requirements"][:30])

[' 1. 具金融或電商產業相關經驗者尤佳\n2. 具備3年以上用戶數據分析經驗\n3. 具備良好商業邏輯能力\n4. 熟悉BI分析軟體(如:Tableau或Power BI)尤佳\n5. 細心、主動、邏輯能力強、態度認真負責 ', ' ●三至五年數據分析相關工作經驗，具金融保險工作經驗尤佳\n●具備良好的邏輯思考能力、溝通能力、整合能力及表達能力\n●熟悉SAS/Python/Access/Excel等\n●具團隊合作精神但也能獨立完成各項專案分析 ', ' 1. 具備數據分析、資料結構高度洞察力及良好的溝通及表達、獨立分析和解決問題及簡報能力。\n2. 具資料倉儲及統計規劃能力與經驗，熟悉SQL/Python/R/Excel等技術及商業智慧工具運用(SAS/Tableau)尤佳。 \n\n ', ' 1.對資料或資料結構具備高度敏感，並具備資料庫結構與擷取能力(基本需求：SQL、Google Analytics、Firebase、Excel)\n2.熟悉大數據處理相關技術與統計之資料相關實務經驗\n3.發掘消費者行為模式與偏好，據此建立數據資料分類，解讀分析的結果，以利行銷策略規劃、應用執行方案與行銷決策建議，分析消費者行為、社群行銷、電子商務.. 等領域\n4.從大量資料產出資料模型、數據分析報表與分析報告、儀表板(基本需求：Google Data Studio建立Dashboards、Python)\n ', ' 1.\t擅溝通、細心、認真負責\r\n2.\t數字邏輯佳，具數據分析相關經驗\r\n3.\t具財產和人身保險業務員資格證照者尤佳\r\n4.\t具有基礎程式能力及CRM分析經驗者優先錄取 ', ' 1.具有3年以上零售業電子商務以及會員數據分析相關經驗。\n2.熟悉Google Analytics/Facebook 等網站數據分析、報表或產出報告。 ', ' 1.大學(含)以上畢業。\n2.2年以上數據分析經驗；零售業、電商、數位產業佳。\n3.熟悉 SQL 語法；有 R / Python 經驗佳。\n4.熟悉 Tableau 操作或其他 BI 軟體。\n5.熟悉資料庫運作方式及管理觀念；有 ETL 經驗佳。\n6.具備會員分析經驗與數據報告撰寫能力，並對數據分析有熱忱。\n7.具備自我學習新技術或新工具的能力，必要時能獨立作業。 ', ' ▍必須

In [14]:
#擅長工具和工作技能
skills_urls = []
for i in trange(len(urls)):
  url = "http:" + urls[i]
  try:
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
      skill = soup.find_all("p", {"class","mb-0 r3"})[0]
      skills_urls.append(skill.find("a")["href"]) #從各職缺子頁再連結到擅長工具和工作技能的頁面
    except:
      #skill.find("a") == None
      #soup.find_all("p", {"class","mb-0 r3"}) == None
      skills_urls.append([])
    driver.quit()
  except:
    skills_urls.append([])

100%|██████████| 1980/1980 [2:51:31<00:00,  5.20s/it]


In [15]:
jobs_scraper["Skills_list"] = []
for i in trange(len(skills_urls)):
  if len(skills_urls[i])==0:
    jobs_scraper["Skills_list"].append([])
  else:
    url = "http:" + skills_urls[i]
    request = rq.get(url)
    soup = BeautifulSoup(request.text, 'lxml')
    #獲得擅長工具和工作技能
    items = soup.find_all("a",{"class","skillSideBar-list__item"})
    skills_list = []
    for item in items:
      skills_list.append(item["data-gtm-sidebar"])
    jobs_scraper["Skills_list"].append(skills_list)

100%|██████████| 1980/1980 [1:14:16<00:00,  2.25s/it]


In [16]:
print(len(jobs_scraper["Skills_list"]))

1980


In [ ]:
print(jobs_scraper["Skills_list"][:30])

[['工具-Python', '工具-R', '工具-MS SQL', '工具-Excel', '工具-PowerPoint', '工具-Power BI', '工具-Tableau', '技能-BI分析', '技能-數據分析', '技能-行銷策略規劃', '技能-競爭者分析', '技能-行銷活動', '技能-使用者研究', '技能-邏輯能力', '技能-分析報告撰寫', '技能-公司營運', '技能-產品開發', '技能-使用者行為', '語言-英文'], ['工具-Python', '工具-Access', '工具-Excel', '工具-Word', '工具-SAS', '工具-Oracle', '技能-整合能力', '技能-業務管理', '技能-表達能力', '技能-統計分析', '技能-專案分析', '技能-客服', '技能-金融保險', '技能-數據分析', '技能-大數據', '技能-報表製作', '技能-經營管理', '技能-客戶專案', '技能-行為分析', '技能-成本控管', '技能-商品銷售', '語言-英文', '職能-溝通能力', '職能-分析思考'], ['工具-Python', '工具-R', '工具-MS SQL', '工具-Tableau', '工具-SAS', '工具-ETL', '工具-SQL', '工具-Excel', '技能-資料結構', '技能-商業分析', '技能-預測', '技能-提案與簡報技巧', '技能-行銷推廣', '技能-統計', '技能-自動化', '技能-數據分析', '技能-模式分析', '技能-數據統計', '技能-策略分析', '技能-消費者行為', '技能-視覺化', '技能-商業智慧', '技能-數據分析報表', '語言-英文'], ['工具-MS SQL', '工具-Excel', '工具-PowerPoint', '工具-Word', '工具-Google Analytics', '工具-Firebase', '工具-Google Data Studio', '工具-Python', '工具-SQL', '技能-資料結構', '技能-跨部門溝通', '技能-營運', '技能-專案規劃與執行', '技能-社群行銷', '技能-統計', '技能-數據分析', '技能-大數據處理', '技能-消費

In [ ]:
#工作內容
jobs_scraper["Job_description"] = []
for i in trange(len(urls)):
  url = "http:" + urls[i]
  request = rq.get(url)
  soup = BeautifulSoup(request.text, 'lxml')
  try:
    jobs_scraper["Job_description"].append(soup.find_all("meta",{"property":"og:description"})[0]["content"])
  except:
    jobs_scraper["Job_description"].append([])
  time.sleep(1)

100%|██████████| 20/20 [00:29<00:00,  1.49s/it]


In [ ]:
print(len(jobs_scraper["Job_description"]))
print(jobs_scraper["Job_description"][:30])

20
['【工作內容】台北市信義區 - ●建立推廣分析架構並提出建議，以透過數據分析提升對AG及客戶之洞察並制定後續業務推動行動方案\n●檢視並分析各項商品推動動態，以建立推動策略及SOP，以達業績目標\n●執行資源整合及獎勵推廣簡報製作，以利…。薪資：月薪38,000元以上。職務類別：統計精算人員、市場調查／市場分析、統計學研究員。休假制度：週休二日。可上班日：不限。工作性質：全職。主動應徵、找工作，請上 104 人力銀行投遞履歷。', '【工作內容】台北市信義區 - 主要職務內容為運用公司客戶及業務員行為資料庫進行各項業務數據分析，協助經營管理階層掌握重要業務數據以擬定業務發展策略及各項業務提升計劃\n●運用big data及統計分析方法建置業務員行為分析模型以協助…。薪資：待遇面議（經常性薪資達4萬元或以上）。職務類別：市場調查／市場分析、統計精算人員。休假制度：週休二日。可上班日：不限。工作性質：全職。主動應徵、找工作，請上 104 人力銀行投遞履歷。', '【工作內容】義大利 - 工作內容：\n數據清洗、數據建模、數據分析、趨勢預測。\n\n職位要求：\n1. 深度熟悉Excel操作,熟練掌握SPSS\\R語言\\Python建模能力尤佳。\n2. 精通統計理論相關知識,並有數據分析實際經驗…。薪資：待遇面議（經常性薪資達4萬元或以上）。職務類別：軟體設計工程師、Internet程式設計師、網頁設計師。休假制度：依公司規定。可上班日：不限。工作性質：全職。主動應徵、找工作，請上 104 人力銀行投遞履歷。', '【工作內容】新北市新店區 - 1.整理編寫每週業務數據分析報告。\n2.專案需求分析、數據分析、商業分析和數據採擷模型等。\n3.提供業務部門數據分析報告，與業務部門合作開展業務專題分析。。薪資：待遇面議（經常性薪資達4萬元或以上）。職務類別：市場調查／市場分析、統計精算人員、財務分析／財務人員。休假制度：週休二日。可上班日：一個月內。工作性質：全職。主動應徵、找工作，請上 104 人力銀行投遞履歷。', '【工作內容】台北市中山區 - 工作職掌：\r\n1.以幫助客戶發現業務問題、發揮數據價值為導向，能深入理解客戶業務需求與痛點，確定客戶業務問題與目標，並給予有效的數據解決方案；\r\n2.從客戶業務需求場景出發，指導/協助客戶完成多維度指…。薪

In [53]:
#創造pandas dataframe
df = pd.DataFrame(jobs_scraper)
df.head()

,Salary,Skills_list,Position,Company
0,"月薪64,000~78,000元",[],數據分析師,星威嘉數位媒體顧問有限公司
1,待遇面議,"[工具-Python, 工具-R, 工具-MS SQL, 工具-Excel, 工具-Powe...",數據分析師,王牌數位創新股份有限公司
2,"月薪40,000~60,000元","[工具-Tableau, 工具-Google Analytics, 工具-Power BI,...",數據分析師,庫柏資訊軟體股份有限公司
3,待遇面議,"[工具-Python, 工具-R, 工具-MS SQL, 工具-MySQL, 工具-PLSQ...",數據分析師,凡谷興業有限公司
4,待遇面議,"[工具-Python, 工具-R, 工具-MySQL, 工具-Excel, 工具-Googl...",數據分析師,台北神策諮詢顧問有限公司


In [54]:
df.to_excel("jobs_scraper.xlsx")

In [ ]:
#Data cleaning (薪資, 工具)

薪資regex

In [19]:
import re

In [20]:
df.Salary.describe()

count     1980
unique     268
top       待遇面議
freq       852
Name: Salary, dtype: object

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1980 entries, 0 to 1979
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Salary       1980 non-null   object
 1   Skills_list  1980 non-null   object
dtypes: object(2)
memory usage: 31.1+ KB


In [22]:
def payed_interval(x):
  salary = re.compile("(\w薪)")
  if salary.search(x) != None:
    return salary.search(x).group()
  else:
    return x 

In [23]:
df.Salary.apply(payed_interval).value_counts()

月薪                   1055
待遇面議                  852
時薪                     41
年薪                     22
日薪                      3
論件計酬0元                  2
論件計酬300~15,000元         2
論件計酬1,000~20,000元       2
論件計酬2,000元以上            1
Name: Salary, dtype: int64

In [24]:
#以regex抓出薪資數字
#只計算支付月薪的薪資
def find_salary(x):
  if "月薪" in x:
    salary = re.compile("(\d{2},\d{3})")
    return salary.findall(x)
  else:
    return 0

In [25]:
#apply在dataframe
df["Salary_extracted"] = df.Salary.apply(find_salary)

In [26]:
df.head(20)

,Salary,Skills_list,Salary_extracted
0,"月薪64,000~78,000元",[],"[64,000, 78,000]"
1,待遇面議,"[工具-Python, 工具-R, 工具-MS SQL, 工具-Excel, 工具-Powe...",0
2,"月薪40,000~60,000元","[工具-Tableau, 工具-Google Analytics, 工具-Power BI,...","[40,000, 60,000]"
3,待遇面議,"[工具-Python, 工具-R, 工具-MS SQL, 工具-MySQL, 工具-PLSQ...",0
4,待遇面議,"[工具-Python, 工具-R, 工具-MySQL, 工具-Excel, 工具-Googl...",0
5,待遇面議,"[工具-Python, 工具-R, 工具-Tableau, 工具-Power BI, 工具-...",0
6,待遇面議,"[工具-Access, 工具-Excel, 工具-PowerPoint, 工具-Word, ...",0
7,"月薪29,000~31,000元","[工具-Excel, 工具-Outlook, 工具-PowerPoint, 工具-Word,...","[29,000, 31,000]"
8,"月薪32,000~60,000元","[工具-C, 工具-C#, 工具-C++, 工具-Java, 工具-Matlab, 工具-P...","[32,000, 60,000]"
9,"月薪35,000~45,000元",[],"[35,000, 45,000]"


In [ ]:
"""
○1. 去掉','
○2. 將字串轉為數值
○3. 在有空list、一個值、和兩個值的情況下，計算用兩個值的最小值和平均值計算平均值
○4. 只計算月薪
"""

In [27]:
def avg_by_min(x):
  #除了薪資面議的資料
  if x != 0:
    #兩個值的資料取第一個
    #去掉","並轉換為數值
    return int(x[0].replace(",",""))
  else:
    return 0

In [28]:
df["Avg_by_min"] = df.Salary_extracted.apply(avg_by_min)

In [45]:
#有取到值的資料筆數(確定抓到的值都來自月薪)
len(df[df.Avg_by_min != 0])

1051

In [46]:
def test_avg_by_min(x):
  #除了薪資面議的資料
  if x != 0:
    #兩個值的資料取第一個
    #去掉","並轉換為數值
    return 1
  else:
    return 0

In [47]:
df.Salary_extracted.apply(test_avg_by_min).sum()
# 有四個月薪的值抓到0

1055

In [30]:
#計算平均值
sum_of_AvgByMin = df.Avg_by_min.sum()
print(f"數據分析相關工作的平均月薪為: {round(sum_of_AvgByMin/len(df[df.Avg_by_min != 0]),0)}元")

數據分析相關工作的平均月薪為: 33145.0元


In [31]:
def avg_by_avg(x):
  #除了薪資面議的資料
  if x != 0:
    #只有一個值(以上)
    if len(x) == 1:
      return int(x[0].replace(",",""))
    #有兩個值(區間)，取平均值  
    elif len(x) == 2:
      return (int(x[0].replace(",",""))+int(x[1].replace(",","")))/2
  else:
    return 0

In [32]:
df["Avg_by_avg"] = df.Salary_extracted.apply(avg_by_avg)

In [33]:
#有取到值的資料筆數(確定抓到的值都來自月薪)
len(df[df.Avg_by_avg != 0])

1051

In [34]:
#計算平均值
sum_of_AvgByAvg = df.Avg_by_avg.sum()
print(f"數據分析相關工作的平均月薪為: {round(sum_of_AvgByAvg/len(df[df.Avg_by_avg != 0]),0)}元")

數據分析相關工作的平均月薪為: 38046.0元


擅長技能

In [35]:
df_SkillLists = df.copy()
df_SkillLists.head()

,Salary,Skills_list,Salary_extracted,Avg_by_min,Avg_by_avg
0,"月薪64,000~78,000元",[],"[64,000, 78,000]",64000,71000.0
1,待遇面議,"[工具-Python, 工具-R, 工具-MS SQL, 工具-Excel, 工具-Powe...",0,0,0.0
2,"月薪40,000~60,000元","[工具-Tableau, 工具-Google Analytics, 工具-Power BI,...","[40,000, 60,000]",40000,50000.0
3,待遇面議,"[工具-Python, 工具-R, 工具-MS SQL, 工具-MySQL, 工具-PLSQ...",0,0,0.0
4,待遇面議,"[工具-Python, 工具-R, 工具-MySQL, 工具-Excel, 工具-Googl...",0,0,0.0


In [36]:
# 確定項目數量
df_SkillLists.Skills_list.apply(lambda x: len(x)).sum()

23280

In [37]:
# list裡的每一個項目都獨立成一個row，後續以.value.value_counts()計算各技能的數量
df_SkillLists_expanded = df_SkillLists.Skills_list.apply(pd.Series).reset_index().melt(id_vars="index").dropna()[["index", "value"]].reset_index().copy()

In [38]:
print(f"There are {len(df_SkillLists_expanded)} items in the skills list.")
df_SkillLists_expanded.head(10)

There are 23280 items in the skills list.


,level_0,index,value
0,1,1,工具-Python
1,2,2,工具-Tableau
2,3,3,工具-Python
3,4,4,工具-Python
4,5,5,工具-Python
5,6,6,工具-Access
6,7,7,工具-Excel
7,8,8,工具-C
8,10,10,工具-Excel
9,12,12,工具-Google Analytics


In [39]:
df_SkillLists_expanded.value.value_counts()[:20]

技能-數據分析                949
工具-Excel               739
語言-英文                  609
工具-PowerPoint          593
工具-Word                578
技能-操作                  292
工具-Google Analytics    280
工具-Python              242
工具-Facebook            222
技能-社群經營                215
工具-Outlook             212
技能-提案與簡報技巧             206
技能-商品銷售                188
技能-FB經營                186
技能-營運                  169
技能-廣告企劃案╱文案撰寫          165
工具-Photoshop           160
技能-行銷策略擬定              141
技能-品牌行銷管理              119
工具-Illustrator         117
Name: value, dtype: int64

In [40]:
def print_ranked_skills(ranked_skills, list_len):
  for i in range(list_len):
    print(f"Number {i+1} requested skill is \"{ranked_skills.index[i]}\", which has been mentioned {ranked_skills[i]} times.")

In [41]:
Ranked_skills = df_SkillLists_expanded.value.value_counts()
print_ranked_skills(Ranked_skills, 20)

Number 1 requested skill is "技能-數據分析", which has been mentioned 949 times.
Number 2 requested skill is "工具-Excel", which has been mentioned 739 times.
Number 3 requested skill is "語言-英文", which has been mentioned 609 times.
Number 4 requested skill is "工具-PowerPoint", which has been mentioned 593 times.
Number 5 requested skill is "工具-Word", which has been mentioned 578 times.
Number 6 requested skill is "技能-操作", which has been mentioned 292 times.
Number 7 requested skill is "工具-Google Analytics", which has been mentioned 280 times.
Number 8 requested skill is "工具-Python", which has been mentioned 242 times.
Number 9 requested skill is "工具-Facebook", which has been mentioned 222 times.
Number 10 requested skill is "技能-社群經營", which has been mentioned 215 times.
Number 11 requested skill is "工具-Outlook", which has been mentioned 212 times.
Number 12 requested skill is "技能-提案與簡報技巧", which has been mentioned 206 times.
Number 13 requested skill is "技能-商品銷售", which has been mentioned 188 t

In [42]:
#根據類別做排序
df_split_skills = df_SkillLists_expanded.value.apply(lambda x: x.split("-",1)).apply(pd.Series)

In [43]:
df_split_skills.columns = ["Types","Names"]
df_split_skills.head()

,Types,Names
0,工具,Python
1,工具,Tableau
2,工具,Python
3,工具,Python
4,工具,Python


In [44]:
Ranked_skills = df_split_skills[df_split_skills.Types == "工具"].Names.value_counts()
print_ranked_skills(Ranked_skills, 10)

Number 1 requested skill is "Excel", which has been mentioned 739 times.
Number 2 requested skill is "PowerPoint", which has been mentioned 593 times.
Number 3 requested skill is "Word", which has been mentioned 578 times.
Number 4 requested skill is "Google Analytics", which has been mentioned 280 times.
Number 5 requested skill is "Python", which has been mentioned 242 times.
Number 6 requested skill is "Facebook", which has been mentioned 222 times.
Number 7 requested skill is "Outlook", which has been mentioned 212 times.
Number 8 requested skill is "Photoshop", which has been mentioned 160 times.
Number 9 requested skill is "MS SQL", which has been mentioned 117 times.
Number 10 requested skill is "Illustrator", which has been mentioned 117 times.
